In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import re  # Добавляем библиотеку для работы с регулярными выражениями

In [2]:
# URL-адрес для парсинга
tender_url = "https://rostender.info/region/permskij-kraj?branch12=12&active_filter=YES"

In [3]:
# Отправка GET-запроса
web_response = requests.get(tender_url)

# Создание объекта BeautifulSoup
soup_object = BeautifulSoup(web_response.text, "html.parser")

# Извлечение всех статей
tender_articles = soup_object.findAll('article', class_='tender-row row')

In [4]:
# Функция для извлечения текста из тега
def get_text_from_tag(tag):
    return tag.get_text(strip=True)

# Функция для преобразования даты
def change_date_format(date_string, format='%d.%m.%y'):
    return datetime.strptime(date_string, format)

# Функция для извлечения информации о тендере
def get_tender_details(article):
    tender_id_string = get_text_from_tag(article.find(class_='tender__number'))
    tender_id = int(re.findall(r'\d+', tender_id_string)[0])  # Извлекаем числа из строки
    tender_title = get_text_from_tag(article.find(class_='description tender-info__description tender-info__link'))
    start_date_string = get_text_from_tag(article.find(class_='tender__date-start'))
    tender_start_date = change_date_format(re.findall(r'\d{2}.\d{2}.\d{2}', start_date_string)[0])  # Извлекаем дату из строки
    end_date_string = get_text_from_tag(article.find(class_='tender__countdown-text').find(class_='black'))
    tender_end_date = change_date_format(re.findall(r'\d{2}.\d{2}.\d{4}', end_date_string)[0], format='%d.%m.%Y')  # Извлекаем дату из строки
    tender_location = get_text_from_tag(article.find(class_='line-clamp line-clamp--n3')).replace(";", ", ")
    tender_price_string = get_text_from_tag(article.find(class_='starting-price__price starting-price--price'))
    tender_price = None if tender_price_string == '—' else int(''.join(re.findall(r'\d+', tender_price_string)))  # Удаляем пробелы и извлекаем числа из строки
    tender_industries = ', '.join([get_text_from_tag(li) for li in article.find('ul', class_='list-branches__ul').findAll('li')])

    # Вывод информации о тендере
    print(f"Номер тендера: {tender_id}")
    print(f"Наименование тендера: {tender_title}")
    print(f"Дата начала тендера: {tender_start_date}")
    print(f"Дата окончания тендера: {tender_end_date}")
    print(f"Геолокация тендера: {tender_location}")
    print(f"Цена тендера в рублях: {tender_price}")
    print(f"Отрасли тендера: {tender_industries}")
    print("\n")

    return [tender_id, tender_title, tender_start_date, tender_end_date, tender_location, tender_price, tender_industries]


In [5]:
# Извлечение информации о всех тендерах
all_tenders_data = [get_tender_details(article) for article in tender_articles]

Номер тендера: 72469413
Наименование тендера: Поставка носителей ключевой и идентификационной информации
Дата начала тендера: 2023-12-09 00:00:00
Дата окончания тендера: 2023-12-12 00:00:00
Геолокация тендера: г. Добрянка
Цена тендера в рублях: 356900
Отрасли тендера: Оборудование для защиты информации


Номер тендера: 30578133866
Наименование тендера: ОКАЗАНИЕ УСЛУГ СВЯЗИ "ИНТЕРНЕТ ДОМ. ГИ БИЗНЕС"
Дата начала тендера: 2023-12-09 00:00:00
Дата окончания тендера: 2023-12-09 00:00:00
Геолокация тендера: г. Пермь
Цена тендера в рублях: 237600
Отрасли тендера: Услуги Интернет, передачи данных, местной телефонной связи


Номер тендера: 72466127
Наименование тендера: Выполнение монтажных, пуско-наладочных работ и оказание услуг по информационной поддержке РОКС в Пермском крае
Дата начала тендера: 2023-12-09 00:00:00
Дата окончания тендера: 2023-12-14 00:00:00
Геолокация тендера: Пермский край
Цена тендера в рублях: 11540960
Отрасли тендера: Монтаж, ремонт и обслуживание телекоммуникационного

In [6]:
# Создание DataFrame
tenders_dataframe = pd.DataFrame(all_tenders_data, columns=["Номер", "Наименование", "Дата начала", "Дата окончания", "Геолокация", "Цена в рублях", "Отрасли"])

# Вывод DataFrame
tenders_dataframe

,Номер,Наименование,Дата начала,Дата окончания,Геолокация,Цена в рублях,Отрасли
0,72469413,Поставка носителей ключевой и идентификационно...,2023-12-09,2023-12-12,г. Добрянка,356900.0,Оборудование для защиты информации
1,30578133866,"ОКАЗАНИЕ УСЛУГ СВЯЗИ ""ИНТЕРНЕТ ДОМ. ГИ БИЗНЕС""",2023-12-09,2023-12-09,г. Пермь,237600.0,"Услуги Интернет, передачи данных, местной теле..."
2,72466127,"Выполнение монтажных, пуско-наладочных работ и...",2023-12-09,2023-12-14,Пермский край,11540960.0,"Монтаж, ремонт и обслуживание телекоммуникацио..."
3,72464831,Оказание услуг по проектированию средств защит...,2023-12-09,2023-12-15,"Спасский район, поселок Приволжский, Тукаевс...",10000000.0,Услуги в области защиты информации
4,72458435,Телефонный аппарат,2023-12-08,2023-12-08,г. Чернушка,30000.0,"Телекоммуникационное оборудование и материалы,..."
5,72467925,Оказание услуг интернета на ФАПы ГБУЗ ПК НРБ,2023-12-08,2023-12-19,г. Нытва,489327.0,"Услуги Интернет, передачи данных, местной теле..."
6,72467909,Оказание услуг доступа интернет ГБУЗ ПК НРБ,2023-12-08,2023-12-21,г. Нытва,885360.0,"Услуги Интернет, передачи данных, местной теле..."
7,30578093880,Оказание услуг по предоставлению услуг связи (...,2023-12-08,2023-12-08,Пермский край,115200.0,"Услуги Интернет, передачи данных, местной теле..."
8,30578110864,Оказание услуг общедоступной электрической связи,2023-12-08,2023-12-08,"Кочевский район, село Кочево",270000.0,"Услуги Интернет, передачи данных, местной теле..."
9,72464789,Закупка компьютерного оборудования в региональ...,2023-12-08,2023-12-15,"г. Самара, г. Ростов-на-Дону, Город Санкт-Пе...",NaN,"Инструменты, Торговое и складское оборудование..."
